# 3 - Calibration

In [15]:
%matplotlib inline
import matplotlib as plt
plt.rcParams['figure.figsize'] = [21, 8]
plt.rcParams['figure.dpi'] = 100

In [4]:
from bsp.core.io import load_study
study = load_study("data/prueba_fixed.bsp")
initial_calibration = study[0]
final_calibration = study[-1]

In [13]:
from typing import Iterator
from numpy import ndarray, median, mean

from bsp.core import impulses

def _amplitudes(channel: ndarray) -> Iterator[int]:
    for start, end in impulses(channel):
        start_value = channel[start]
        end_value = channel[end]
        amplitude = max(start_value, end_value) - min(start_value, end_value)
        yield amplitude

def _valid_amplitudes(channel: ndarray, tolerance: float=0.2) -> Iterator[int]:
    amplitudes = list(_amplitudes(channel))
    median_amplitude = median(amplitudes)
    delta = median_amplitude * tolerance
    min_value, max_value = median_amplitude - delta, median_amplitude + delta
    for amplitude in amplitudes:
        if min_value <= amplitude <= max_value:
            yield amplitude


def calibration(initial: ndarray, final: ndarray, angle: int=30) -> tuple[float, float]:
    initial_amplitudes = list(_valid_amplitudes(initial))
    final_amplitudes = list(_valid_amplitudes(final))

    initial_mean = mean(initial_amplitudes)
    final_mean = mean(final_amplitudes)

    initial_scale = angle / initial_mean
    final_scale = angle / final_mean

    scale = (initial_scale + final_scale) / 2
    diff = initial_scale / final_scale

    return scale, diff


hor_scale, diff = calibration(
    initial=initial_calibration.horizontal_channel,
    final=final_calibration.horizontal_channel,
    angle=30,
)
print(hor_scale, diff)

0.0021364740046483587 0.9679631783217272
